# Invoice Extraction Evaluation

This notebook evaluates the extraction of data points from PDF invoice documents.

In [1]:
import pandas as pd
import boto3
import requests
import os
import sys
from pathlib import Path
import json

# Add backend directory to Python path
backend_path = str(Path.cwd().parent / 'backend')
if backend_path not in sys.path:
    sys.path.append(backend_path)

from extract import extract_invoice_from_file

In [2]:
# S3 configuration
S3_BUCKET = 'wassim-xelix-data-science'  # Replace with your bucket name
S3_PREFIX = 'ai_assistant/invoice_document_extraction/large_invoice_sample/'  # Replace with your desired prefix

# Initialize S3 client
s3_client = boto3.client('s3')

# Local path for results CSV
LOCAL_RESULTS_CSV_PATH = 'invoice_extraction_results_local.csv'

In [3]:
# Load the sample data
df = pd.read_csv('invoice_sample.csv')
df.head(10)

,id,number,date,due_date,currency_code,gross_amount,vat_amount,net_amount,payment_term_days,name,value,name-2,display_name
0,21214858,CI0205714,2025-02-05,2025-05-05,0,72.00,12.00,60.00,86,UNIVERSAL COMPONENTS UK LTD,07022502A1,misc_ref,MiscRef
1,13899086,126959,2018-02-21,2018-04-30,0,1800.00,300.00,1500.00,12,ACTIVE BRAND CONCEPTS LIMITED (ABC),1304180025,misc_ref,MiscRef
2,14136172,17390213,2018-08-28,2018-09-15,0,87.53,14.59,72.94,16,KLARIUS PRODUCTS LTD,29082018,misc_ref,MiscRef
3,13713597,17935432,2017-12-13,2018-02-28,0,44.83,7.47,37.36,58,FERRARIS PISTON SERVICES LTD,14122017,misc_ref,MiscRef
4,14840064,02746270,2020-01-09,2020-03-31,0,25.56,4.26,21.30,81,FERRARIS PISTON SERVICES LTD,10012020,misc_ref,MiscRef
5,14458465,LHR2416820,2019-03-31,2019-04-30,0,53.40,8.90,44.50,27,DHL INTERNATIONAL (UK) LIMITED,02041904CE,misc_ref,MiscRef
6,20886311,18503991,2024-02-19,2024-03-15,0,758.33,126.39,631.94,24,KLARIUS PRODUCTS LTD,20022024,misc_ref,MiscRef
7,14082600,9580397893,2018-07-25,2018-08-30,0,255.22,42.54,212.68,281,3M UNITED KINGDOM PLC (PAINT),26071803EA,misc_ref,MiscRef
8,14671199,01365581,2019-08-30,2019-10-31,0,73.51,12.25,61.26,60,FERRARIS PISTON SERVICES LTD,31082019,misc_ref,MiscRef
9,15091491,2257170979,2020-07-02,2020-09-30,0,504.90,84.15,420.75,79,ROBERT BOSCH LIMITED,130720027D,misc_ref,MiscRef


In [4]:
def process_invoice_row(row):
    """Process a single invoice row from the dataframe."""
    value = row['value']
    pdf_key = f"{S3_PREFIX}{value}.pdf"
    
    # Check if PDF exists in S3
    try:
        s3_client.head_object(Bucket=S3_BUCKET, Key=pdf_key)
        pdf_exists = True
    except:
        pdf_exists = False
    
    if not pdf_exists:
        # Try to download from the URL
        url = f"https://ecp.e-docsonline.co.uk/Q.aspx?&1=Q/{value}.PDF"
        try:
            response = requests.get(url)
            if response.status_code == 200 and response.headers.get('content-type') == 'application/pdf':
                # Save to S3
                s3_client.put_object(
                    Bucket=S3_BUCKET,
                    Key=pdf_key,
                    Body=response.content
                )
                pdf_exists = True
            else:
                return {"error": f"Could not download PDF from URL for {value}"}
        except Exception as e:
            return {"error": f"Error downloading PDF: {str(e)}"}
    
    if pdf_exists:
        # Download PDF from S3 to temporary file
        temp_path = f"/tmp/{value}.pdf"
        try:
            s3_client.download_file(S3_BUCKET, pdf_key, temp_path)
            # Extract data using backend methods
            result = extract_invoice_from_file(temp_path)
            # Clean up temporary file
            os.remove(temp_path)
            return result
        except Exception as e:
            return {"error": f"Error processing PDF: {str(e)}"}
    
    return {"error": "No PDF available for processing"}

In [ ]:
process_invoice_row(df[:1])

In [14]:
# Define the expected columns and their data types for your results DataFrame
EXPECTED_DTYPES = {
    'original_invoice_id': 'string',
    'extraction_status': 'string',
    'error_message': 'string',
    'extracted_number': 'string',
    'extracted_po_number': 'string',
    'extracted_amount': 'Float64',
    'extracted_tax_amount': 'Float64',
    'extracted_currency_code': 'string',
    'extracted_date': 'string',
    'extracted_due_date': 'string',
    'extracted_payment_term_days': 'Int64',
    'extracted_vendor': 'string',
    'extracted_line_items': 'string'
}
EXPECTED_COLUMNS = list(EXPECTED_DTYPES.keys())

# --- Load existing results or initialize ---
if os.path.exists(LOCAL_RESULTS_CSV_PATH):
    print(f"Loading existing results from {LOCAL_RESULTS_CSV_PATH}...")
    try:
        # More comprehensive list of NA values for robust parsing
        na_values_list = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan',
                           '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NA', 'NULL', 'NaN', 'None', 'nan', 'null']
        master_results_df = pd.read_csv(
            LOCAL_RESULTS_CSV_PATH,
            dtype=EXPECTED_DTYPES,
            keep_default_na=True,
            na_values=na_values_list
        )

        for col, dtype_str in EXPECTED_DTYPES.items():
            if col not in master_results_df.columns:
                master_results_df[col] = pd.NA
            try:
                master_results_df[col] = master_results_df[col].astype(EXPECTED_DTYPES[col])
            except Exception as e:
                print(f"Warning: Could not cast column {col} to {EXPECTED_DTYPES[col]} on load: {e}. Leaving as is or with NAs.")


        processed_invoice_ids = set(master_results_df['original_invoice_id'].dropna().astype(str).tolist())
        print(f"Loaded {len(processed_invoice_ids)} previously processed invoice IDs.")
    except pd.errors.EmptyDataError:
        print(f"{LOCAL_RESULTS_CSV_PATH} is empty. Initializing fresh DataFrame.")
        master_results_df = pd.DataFrame(columns=EXPECTED_COLUMNS).astype(EXPECTED_DTYPES)
        processed_invoice_ids = set()
    except Exception as e:
        print(f"Error loading {LOCAL_RESULTS_CSV_PATH}: {e}. Initializing fresh DataFrame.")
        master_results_df = pd.DataFrame(columns=EXPECTED_COLUMNS).astype(EXPECTED_DTYPES)
        processed_invoice_ids = set()
else:
    print(f"No existing local results file found. Initializing fresh DataFrame.")
    master_results_df = pd.DataFrame(columns=EXPECTED_COLUMNS).astype(EXPECTED_DTYPES)
    processed_invoice_ids = set()

# --- Process invoices ---
sample_size = len(df)
invoices_to_process = df.head(sample_size)

print(f"\nStarting processing. Total invoices to consider: {len(invoices_to_process)}")

# Add counter for processed invoices
processed_count = 0
total_to_process = len(invoices_to_process)

for index, row in invoices_to_process.iterrows():
    invoice_id = str(row['id'])

    processed_count += 1
    print(f"Processing invoice {processed_count} of {total_to_process} (ID: {invoice_id})...")

    if invoice_id in processed_invoice_ids:
        print(f"Invoice ID: {invoice_id} already processed. Skipping.")
        continue

    extraction_output = process_invoice_row(row)

    flat_result_for_row = {col: pd.NA for col in EXPECTED_COLUMNS}
    flat_result_for_row['original_invoice_id'] = invoice_id

    if extraction_output and 'error' in extraction_output:
        flat_result_for_row['extraction_status'] = 'error'
        flat_result_for_row['error_message'] = extraction_output['error'] if extraction_output['error'] else pd.NA
    elif extraction_output and 'invoices' in extraction_output and \
         isinstance(extraction_output['invoices'], list) and len(extraction_output['invoices']) > 0:
        invoice_details = extraction_output['invoices'][0]
        flat_result_for_row['extraction_status'] = 'success'

        for field_name_df in EXPECTED_COLUMNS:
            if field_name_df in ['original_invoice_id', 'extraction_status', 'error_message', 'extracted_line_items']:
                continue
            
            source_key = field_name_df.replace('extracted_', '') # Key in invoice_details dict
            raw_value = invoice_details.get(source_key)

            # Treat empty strings and None as NA, but preserve 0 values
            if raw_value == "" or raw_value is None:
                flat_result_for_row[field_name_df] = pd.NA
            else:
                flat_result_for_row[field_name_df] = raw_value
        
        line_items_data = invoice_details.get('line_items', [])
        if not line_items_data: # Handles None or empty list []
            flat_result_for_row['extracted_line_items'] = pd.NA
        else:
            flat_result_for_row['extracted_line_items'] = json.dumps(line_items_data)
            if flat_result_for_row['extracted_line_items'] == '[]': # Convert empty JSON array string to NA
                flat_result_for_row['extracted_line_items'] = pd.NA


    else:
        flat_result_for_row['extraction_status'] = 'no_data_extracted'
        flat_result_for_row['error_message'] = 'No invoice data found or unexpected format in extraction_output.'

    new_row_df = pd.DataFrame([flat_result_for_row])
    # Ensure dtypes. This .astype() call is crucial for converting processed values (like "" handled above)
    # into the correct pd.NA representation for nullable types.
    new_row_df = new_row_df[EXPECTED_COLUMNS].astype(EXPECTED_DTYPES)

    master_results_df = pd.concat([master_results_df, new_row_df], ignore_index=True)

    try:
        # Use na_rep to specify how NA values should be written to CSV
        master_results_df.to_csv(LOCAL_RESULTS_CSV_PATH, index=False, na_rep='NA')
        processed_invoice_ids.add(invoice_id)
        print(f"Completed processing for invoice_id: {invoice_id}. Results saved to {LOCAL_RESULTS_CSV_PATH}\n")
    except Exception as e:
        print(f"CRITICAL ERROR: Could not save results to {LOCAL_RESULTS_CSV_PATH} for invoice_id {invoice_id}. Error: {e}")
        print("Stopping further processing. Please check file permissions or disk space.")
        break

print("\n--- All processing complete or stopped. ---")
print(f"Final results are in {LOCAL_RESULTS_CSV_PATH}")

Loading existing results from invoice_extraction_results_local.csv...
Loaded 1164 previously processed invoice IDs.

Starting processing. Total invoices to consider: 2000
Processing invoice 1 of 2000 (ID: 21214858)...
Invoice ID: 21214858 already processed. Skipping.
Processing invoice 2 of 2000 (ID: 13899086)...
Invoice ID: 13899086 already processed. Skipping.
Processing invoice 3 of 2000 (ID: 14136172)...
Invoice ID: 14136172 already processed. Skipping.
Processing invoice 4 of 2000 (ID: 13713597)...
Invoice ID: 13713597 already processed. Skipping.
Processing invoice 5 of 2000 (ID: 14840064)...
Invoice ID: 14840064 already processed. Skipping.
Processing invoice 6 of 2000 (ID: 14458465)...
Invoice ID: 14458465 already processed. Skipping.
Processing invoice 7 of 2000 (ID: 20886311)...
Invoice ID: 20886311 already processed. Skipping.
Processing invoice 8 of 2000 (ID: 14082600)...
Invoice ID: 14082600 already processed. Skipping.
Processing invoice 9 of 2000 (ID: 14671199)...
Invoic

Processing PDF with 1 pages...
Processing 1 image(s) with AWS Bedrock...
Found 1 invoices


Completed processing for invoice_id: 20637424. Results saved to invoice_extraction_results_local.csv

Processing invoice 1166 of 2000 (ID: 14271985)...
Completed processing for invoice_id: 14271985. Results saved to invoice_extraction_results_local.csv

Processing invoice 1167 of 2000 (ID: 19871094)...


Processing PDF with 1 pages...
Processing 1 image(s) with AWS Bedrock...
Found 1 invoices


Completed processing for invoice_id: 19871094. Results saved to invoice_extraction_results_local.csv

Processing invoice 1168 of 2000 (ID: 21081393)...


...

Processing PDF with 1 pages...
Processing 1 image(s) with AWS Bedrock...
Found 1 invoices


Completed processing for invoice_id: 18865604. Results saved to invoice_extraction_results_local.csv

Processing invoice 1999 of 2000 (ID: 20388155)...
Completed processing for invoice_id: 20388155. Results saved to invoice_extraction_results_local.csv

Processing invoice 2000 of 2000 (ID: 14802583)...


Processing PDF with 2 pages...
Processing 2 image(s) with AWS Bedrock...


Completed processing for invoice_id: 14802583. Results saved to invoice_extraction_results_local.csv


--- All processing complete or stopped. ---
Final results are in invoice_extraction_results_local.csv


Found 1 invoices


In [15]:
successful_results = results_df[results_df['extraction_status'] == 'success'].copy()

,original_invoice_id,extraction_status,error_message,extracted_number,extracted_po_number,extracted_amount,extracted_tax_amount,extracted_currency_code,extracted_date,extracted_due_date,extracted_payment_term_days,extracted_vendor,extracted_line_items
0,21214858,success,<NA>,CI0205714,21182,72.0,12.0,GBP,2025-02-05,<NA>,<NA>,Universal Components UK Ltd,"[{""description"": ""STGO CATEGORY 1/2/3 KIT WI"",..."
1,13899086,success,<NA>,126959,<NA>,1800.0,300.0,GBP,2018-02-21,<NA>,<NA>,ACTIVE BRAND CONCEPTS LTD,"[{""description"": ""TQX Valeting Kit ea"", ""quant..."
2,14136172,error,Could not download PDF from URL for 29082018,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,13713597,error,Could not download PDF from URL for 14122017,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,14840064,error,Could not download PDF from URL for 10012020,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,15644179,error,Failed to extract invoice data,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1996,20885052,success,<NA>,9581978555,5000026383D,1934.26,322.38,GBP,2024-02-16,2024-04-16,60,3M,"[{""description"": ""06309, Mask Tape 2328, 24mmx..."
1997,18865604,success,<NA>,432261,256110,11.26,1.88,GBP,2020-08-25,<NA>,<NA>,POWERFLEX,"[{""description"": ""UPPER ENGINE MOUNT INSERT"", ..."
1998,20388155,error,Could not download PDF from URL for 05072022,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [18]:
sum(master_results_df.extraction_status == 'success')

np.int64(1000)

In [20]:
# Load the datasets
results_df = pd.read_csv('invoice_extraction_results_local.csv')
sample_df = pd.read_csv('invoice_sample.csv')

# Filter out error cases
successful_results = results_df[results_df['extraction_status'] == 'success'].copy()

print(f"Total results: {len(results_df)}")
print(f"Successful extractions: {len(successful_results)}")
print(f"Error rate: {(len(results_df) - len(successful_results)) / len(results_df) * 100:.1f}%")

Total results: 2000
Successful extractions: 1000
Error rate: 50.0%


In [21]:
# Join the datasets on invoice_id
joined_df = successful_results.merge(
    sample_df, 
    left_on='original_invoice_id', 
    right_on='id', 
    how='inner'
)

print(f"Successfully joined records: {len(joined_df)}")


Successfully joined records: 1000


In [34]:
# Define comparison functions
def get_match_type(ref1, ref2, threshold=1):
    """Improved string matching function with better whitespace handling"""
    if pd.isna(ref1) or pd.isna(ref2) or ref1 == "" or ref2 == "":
        return 'missing'
    
    # Normalize strings - remove spaces around common separators
    def normalize_reference(ref):
        ref = str(ref).strip().upper()
        # Remove spaces around common separators
        ref = ref.replace(' / ', '/')
        ref = ref.replace(' - ', '-')
        ref = ref.replace(' _ ', '_')
        # Remove all spaces for invoice numbers (common in OCR)
        ref = ref.replace(' ', '')
        return ref
    
    str1 = normalize_reference(ref1)
    str2 = normalize_reference(ref2)
    
    # Exact match after normalization
    if str1 == str2:
        return 'exact'
    
    # Simple edit distance for close matches
    if abs(len(str1) - len(str2)) > threshold:
        return 'none'  # Too different in length
    
    # Count character differences (simple approach)
    max_len = max(len(str1), len(str2))
    min_len = min(len(str1), len(str2))
    
    differences = abs(len(str1) - len(str2))  # Length difference
    differences += sum(c1 != c2 for c1, c2 in zip(str1, str2))  # Character differences
    
    if differences <= threshold:
        return 'close'
    
    return 'none'

def compare_invoice_numbers(extracted, original):
    """Compare invoice numbers with exact and close matching"""
    if pd.isna(extracted) or pd.isna(original):
        return 'missing'
    
    match_type = get_match_type(str(extracted), str(original))
    return match_type

def compare_amounts(extracted, original, tolerance=0.01):
    """Compare amounts with small tolerance for floating point differences"""
    if pd.isna(extracted) or pd.isna(original):
        return 'missing'
    
    try:
        diff = abs(float(extracted) - float(original))
        if diff == 0:
            return 'exact'
        elif diff <= tolerance:
            return 'close'
        else:
            return 'none'
    except (ValueError, TypeError):
        return 'none'

def compare_dates(extracted, original):
    """Compare dates (exact match required)"""
    if pd.isna(extracted) or pd.isna(original):
        return 'missing'
    
    # Normalize date formats for comparison
    try:
        extracted_date = pd.to_datetime(str(extracted)).date()
        original_date = pd.to_datetime(str(original)).date()
        
        if extracted_date == original_date:
            return 'exact'
        else:
            return 'none'
    except:
        return 'none'

In [35]:
# Perform comparisons
joined_df['number_match'] = joined_df.apply(
    lambda row: compare_invoice_numbers(row['extracted_number'], row['number']), axis=1
)

joined_df['amount_match'] = joined_df.apply(
    lambda row: compare_amounts(row['extracted_amount'], row['gross_amount']), axis=1
)

joined_df['date_match'] = joined_df.apply(
    lambda row: compare_dates(row['extracted_date'], row['date']), axis=1
)

joined_df['due_date_match'] = joined_df.apply(
    lambda row: compare_dates(row['extracted_due_date'], row['due_date']), axis=1
)

In [36]:
# Calculate accuracy metrics
def calculate_accuracy(match_column):
    """Calculate accuracy metrics for a match column"""
    total = len(match_column)
    exact_matches = (match_column == 'exact').sum()
    close_matches = (match_column == 'close').sum()
    no_matches = (match_column == 'none').sum()
    missing = (match_column == 'missing').sum()
    
    exact_accuracy = exact_matches / total * 100
    total_accuracy = (exact_matches + close_matches) / total * 100
    
    return {
        'total_records': total,
        'exact_matches': exact_matches,
        'close_matches': close_matches,
        'no_matches': no_matches,
        'missing': missing,
        'exact_accuracy': exact_accuracy,
        'total_accuracy': total_accuracy
    }

# Generate accuracy report
fields = ['number_match', 'amount_match', 'date_match', 'due_date_match']
field_names = ['Invoice Number', 'Amount', 'Date', 'Due Date']

print("\n" + "="*60)
print("EXTRACTION ACCURACY REPORT")
print("="*60)

accuracy_results = {}
for field, field_name in zip(fields, field_names):
    accuracy_results[field_name] = calculate_accuracy(joined_df[field])
    
    print(f"\n{field_name}:")
    print(f"  Exact matches: {accuracy_results[field_name]['exact_matches']}/{accuracy_results[field_name]['total_records']} ({accuracy_results[field_name]['exact_accuracy']:.1f}%)")
    print(f"  Close matches: {accuracy_results[field_name]['close_matches']}/{accuracy_results[field_name]['total_records']} ({accuracy_results[field_name]['close_matches']/accuracy_results[field_name]['total_records']*100:.1f}%)")
    print(f"  Total accuracy: {accuracy_results[field_name]['total_accuracy']:.1f}%")
    print(f"  No matches: {accuracy_results[field_name]['no_matches']}")
    print(f"  Missing values: {accuracy_results[field_name]['missing']}")


EXTRACTION ACCURACY REPORT

Invoice Number:
  Exact matches: 945/1000 (94.5%)
  Close matches: 21/1000 (2.1%)
  Total accuracy: 96.6%
  No matches: 34
  Missing values: 0

Amount:
  Exact matches: 988/1000 (98.8%)
  Close matches: 0/1000 (0.0%)
  Total accuracy: 98.8%
  No matches: 12
  Missing values: 0

Date:
  Exact matches: 964/1000 (96.4%)
  Close matches: 0/1000 (0.0%)
  Total accuracy: 96.4%
  No matches: 36
  Missing values: 0

Due Date:
  Exact matches: 269/1000 (26.9%)
  Close matches: 0/1000 (0.0%)
  Total accuracy: 26.9%
  No matches: 303
  Missing values: 428


In [37]:
# Show some examples of mismatches
print("\n" + "="*60)
print("EXAMPLE MISMATCHES")
print("="*60)

for field, field_name in zip(fields, field_names):
    mismatches = joined_df[joined_df[field] == 'none']
    if len(mismatches) > 0:
        print(f"\n{field_name} mismatches (showing first 3):")
        if field == 'number_match':
            cols = ['original_invoice_id', 'extracted_number', 'number']
        elif field == 'amount_match':
            cols = ['original_invoice_id', 'extracted_amount', 'gross_amount']
        elif field == 'date_match':
            cols = ['original_invoice_id', 'extracted_date', 'date']
        elif field == 'due_date_match':
            cols = ['original_invoice_id', 'extracted_due_date', 'due_date']
        
        print(mismatches[cols].head(3).to_string(index=False))


EXAMPLE MISMATCHES

Invoice Number mismatches (showing first 3):
 original_invoice_id extracted_number     number
            13986138           91,624      91624
            14183477       TI18026298 IT18026298
            14686624            4,349       4349

Amount mismatches (showing first 3):
 original_invoice_id  extracted_amount  gross_amount
            14611489             31.09         23.94
            13625772           6264.76       6264.77
            20110841           3877.68       2675.55

Date mismatches (showing first 3):
 original_invoice_id extracted_date       date
            14213413     2018-11-10 2018-10-11
            14328943     2019-02-01 2019-01-02
            13836614     2018-05-03 2018-03-05

Due Date mismatches (showing first 3):
 original_invoice_id extracted_due_date   due_date
            14082600         2018-08-31 2018-08-30
            21196536         2025-02-28 2025-02-13
            14338880         2019-04-10 2019-02-28


In [48]:
mismatches = joined_df[['original_invoice_id', 'extracted_number', 'number']][joined_df["number_match"] == "none"]
mismatches

,original_invoice_id,extracted_number,number
125,13986138,"91,624",91624
195,14183477,TI18026298,IT18026298
258,14686624,"4,349",4349
264,14324051,26920247,318621
295,14220696,029531734 (COPY),029531734COPY
311,13623984,AR01417521,AR/01417521
345,20603634,MINV-204965,M/INV-204965
349,14156552,113120561 (COPY),113120561COPY
367,14370278,V00149816,VI00149816
372,14194381,185190,185190/2
